In [1]:
import pymongo
from pymongo import MongoClient
import docIterators
import sanitisers
import codecs
import json
import sys
import corpus
import gensim
import vect_generators
import numpy as np
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE, MDS
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


In [45]:
dcit = docIterators.BlueberryIter(
    ['mongodb://localhost:27017/','Cherry','Blueberry'],
    iter_type='DOI')
corp = corpus.Corpus('BLUEBERRY',dcit)
sample_size=1000
s=corp.get_sample(sample_size)
model=gensim.models.Word2Vec.load('RaspberryEngine')
gen = vect_generators.BlueberryGenerator(model)
vectors = []
for doc in s:
    vectors.append(gen.get_vector(doc['doc']))
get_means_and_plot(vectors,n_clusters=5,dimension=3)

Building Dictionary
[######### ] 99% 449000

Sampling 1000 random documents
[######### ] 99% 449000



In [44]:
def get_means_and_plot(vecs,n_clusters=10,dimension=2):
    kmeans = KMeans(
        n_clusters=n_clusters,
        init='k-means++',
        max_iter=50000,
        verbose=0,
        tol=0.00000000001)
    cluster_centres =kmeans.fit(vecs).cluster_centers_
    preds=kmeans.predict(vecs)
    t = TSNE(init='pca',n_components=dimension)
    reduced_vecs = t.fit_transform(vecs+list(cluster_centres))
    colors=iter(plt.cm.rainbow(np.linspace(0,1,n_clusters)))
    fig = plt.figure()
    if dimension==3:
        ax = fig.gca(projection='3d')
    else:
        ax = fig.gca()
    for centre in range(n_clusters):
        color=next(colors)
        cluster=[]
        for dim in range(dimension):
            cluster.append([reduced_vecs[i][dim] for i in range(sample_size) if preds[i]==centre])
        if dimension==3:
            ax.scatter(cluster[0],
                cluster[1],
                cluster[2],
                 'o',
                 c=color)
            ax.scatter(reduced_vecs[sample_size+centre][0],
                 reduced_vecs[sample_size+centre][1],
                 reduced_vecs[sample_size+centre][2],
                 '*',
                 c=color)
        else:
            ax.plot(cluster[0],
                cluster[1],
                 'o',
                 c=color,
                 markersize=3)
            ax.plot(reduced_vecs[sample_size+centre][0],
                 reduced_vecs[sample_size+centre][1],
                 '*',
                 c=color,
                 markersize=10)
    plt.show()

In [43]:
vecs=vectors
n_clusters=10
dimension=3
kmeans = KMeans(
        n_clusters=n_clusters,
        init='k-means++',
        max_iter=50000,
        verbose=0,
        tol=0.00000000001)
cluster_centres =kmeans.fit(vecs).cluster_centers_
preds=kmeans.predict(vecs)
t = TSNE(init='pca',n_components=dimension)
reduced_vecs = t.fit_transform(vecs+list(cluster_centres))
colors=iter(plt.cm.rainbow(np.linspace(0,1,n_clusters)))
fig = plt.figure()
if dimension==3:
    ax = fig.gca(projection='3d')
else:
    ax = fig.gca()
for centre in range(n_clusters):
    color=next(colors)
    cluster=[]
    for dim in range(dimension):
        cluster.append([reduced_vecs[i][dim] for i in range(sample_size) if preds[i]==centre])
    if dimension==3:
        ax.scatter(cluster[0],
                cluster[1],
                cluster[2],
                 'o',
                 c=color)
        ax.scatter(reduced_vecs[sample_size+centre][0],
                 reduced_vecs[sample_size+centre][1],
                 reduced_vecs[sample_size+centre][2],
                 '*',
                 c=color)
    else:
        ax.plot(cluster[0],
                cluster[1],
                 'o',
                 c=color,
                 markersize=3)
        ax.plot(reduced_vecs[sample_size+centre][0],
                 reduced_vecs[sample_size+centre][1],
                 '*',
                 c=color,
                 markersize=10)
plt.show()

In [36]:
get_means_and_plot(vectors,dimension=3)

KeyboardInterrupt: 

In [ ]:
ind=0
cl = MongoClient()
cran = cl['Cherry']['raspberry']
blue = cl['Cherry']['Blueberry']
for rec in cran.find({'abstract': {'$exists': True}, '$where': "this.abstract.length>0"}):
    pub=rec['publisher']
    title=rec['title']
    abst=rec[u'abstract']
    source_url=rec['source_url']
    authors=rec['authors']
    affiliations=rec['affiliations']
    date=rec['date']
    journal=rec['journal']
    blue.update_one({'doi':rec['doi']},
                    {'$set':
                        {'publisher':pub,
                        'raw_title':title,
                        'raw_abstract':abst,
                        'source_url':source_url,
                        'date':date,
                        'journal':journal}})
    ind+=1
    if ind%1000==0:
        sys.stdout.write(str(ind))
        sys.stdout.flush()

10002000300040005000600070008000900010000110001200013000140001500016000170001800019000